In [40]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss
import pytorch_lightning as pl
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Set default tensor type to float32
torch.set_default_tensor_type(torch.FloatTensor)

# Load and preprocess data
df = pd.read_csv('real_time_cleaned.csv')
df['block_time'] = pd.to_datetime(df['block_time'])
df = df.sort_values('block_time')

# Feature Selection
features = [
    'block_height', 'tx_count', 'mempool_size_mb', 'max_fee_rate', 'avg_fee_rate',
    'median_fee_rate', 'fee_rate_10th', 'fee_rate_90th', 'fee_rate_std',
    'difficulty', 'hash_rate', 'mempool_min_fee', 'total_fee', 'mempool_usage',
    'transaction_count', 'block_weight', 'block_version', 'block_interval',
    'bitcoin_price_usd', 'hist_low_fee_ratio',
    'hist_med_fee_ratio', 'hist_high_fee_ratio', 'hist_fee_diversity'
]

# Prepare data for TFT
df['time_idx'] = range(len(df))
df['group'] = 'group_0'  # Single group as a string

target = 'block_median_fee_rate'
max_encoder_length = 24  # Past 24 observations
max_prediction_length = 10 # Predict next 10 block

# Remove rows with NaN or infinite values in the target or features
df = df.replace([np.inf, -np.inf], np.nan)
print("Null values in dataset:")
print(df.isnull().sum())
df = df.dropna(subset=[target] + features)
print(f"Shape after dropping NaNs: {df.shape}")

# Scale features and ensure float32 dtype
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features]).astype(np.float32)
df[target] = df[target].astype(np.float32)

# Convert 'group' to categorical
df['group'] = df['group'].astype('category')

# Create TimeSeriesDataSet with explicit configuration
tsdataset = TimeSeriesDataSet(
    df,
    time_idx='time_idx',
    target=target,
    group_ids=['group'],
    min_encoder_length=max_encoder_length // 2,
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=['group'],
    static_reals=[],
    time_varying_known_reals=['time_idx'] + features,
    time_varying_unknown_reals=[target],
    target_normalizer=GroupNormalizer(
        groups=['group'], transformation="softplus"
    ),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=False,
)

# Custom collate function to avoid NoneType values
def collate_fn(batch):
    batch = list(filter(lambda x: x is not None, batch))  # Remove NoneType items
    return torch.utils.data.dataloader.default_collate(batch)

# Create dataloaders
batch_size = 32
train_dataloader = DataLoader(
    tsdataset, 
    batch_size=batch_size, 
    shuffle=False,
    collate_fn=collate_fn  # Use the custom collate function
)

# Define the Temporal Fusion Transformer model
class BitcoinFeePredictionModel(pl.LightningModule):
    def __init__(self, tsdataset):
        super().__init__()
        self.model = TemporalFusionTransformer.from_dataset(
            tsdataset,
            learning_rate=0.03,
            hidden_size=32,
            attention_head_size=2,
            dropout=0.1,
            hidden_continuous_size=16,
            loss=QuantileLoss(),
            optimizer="adamw",
            reduce_on_plateau_patience=4,
        )
    
    def forward(self, x):
        output = self.model(x)
        return output
    
    def training_step(self, batch, batch_idx):
        loss = self.model.training_step(batch, batch_idx)
        return loss
    
    def configure_optimizers(self):
        return self.model.configure_optimizers()

# Initialize the model
model = BitcoinFeePredictionModel(tsdataset)

# Train the model
trainer = pl.Trainer(
    max_epochs=30,
    accelerator='mps',  # Use MPS for Apple Silicon
    devices=1,
)

# Debugging step to check the dataloader before fitting
for batch_idx, batch in enumerate(train_dataloader):
    print(f"Batch {batch_idx} content: {batch}")
    if batch_idx == 0:
        break  # Check only the first batch

# Train the model
trainer.fit(model, train_dataloader)

# Prepare data for prediction
last_data = df.iloc[-max_encoder_length:].copy()
last_data['time_idx'] = range(last_data['time_idx'].min(), last_data['time_idx'].max() + 1)

# Make predictions
model.eval()
predictions = model.model.predict(last_data, mode="raw", return_x=True)

# Extract the predicted values
predicted_values = predictions.output.prediction.cpu().numpy()

# Plot results
plt.figure(figsize=(12, 6))
y_true = df[target].iloc[-max_prediction_length:].values
plt.plot(range(len(y_true)), y_true, label='Actual')
plt.plot(range(len(predicted_values)), predicted_values[:, 0, 0], label='Predicted')
plt.title('Actual vs Predicted Bitcoin Fee Rates')
plt.xlabel('Time Index')
plt.ylabel('Fee Rate')
plt.legend()
plt.show()

# Calculate performance metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_true, predicted_values[:, 0, 0])
mse = mean_squared_error(y_true, predicted_values[:, 0, 0])
rmse = np.sqrt(mse)
r2 = r2_score(y_true, predicted_values[:, 0, 0])

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared Score: {r2}")


Null values in dataset:
block_time               0
block_height             0
tx_count                 0
mempool_size_mb          0
max_fee_rate             0
avg_fee_rate             0
median_fee_rate          0
fee_rate_10th            0
fee_rate_90th            0
fee_rate_std             0
difficulty               0
hash_rate                0
mempool_min_fee          0
total_fee                0
mempool_usage            0
transaction_count        0
block_weight             0
block_version            0
block_interval           0
block_median_fee_rate    0
mempool_fee_histogram    0
bitcoin_price_usd        0
hist_low_fee_ratio       0
hist_med_fee_ratio       0
hist_high_fee_ratio      0
hist_fee_diversity       0
time_idx                 0
group                    0
dtype: int64
Shape after dropping NaNs: (2632, 28)


/Users/jiangqinma/miniconda3/envs/bitcoin/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/Users/jiangqinma/miniconda3/envs/bitcoin/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>